## Machine-Learning
---
We will be using three types of models to predict prices of listing, and also using three types of optimization methods to improve the prediction accuracy.

### Tabe of Contents
1. [Modelling](#model)
2. [Optimization](#optimize)
3. [Summary](#summary)

In [1]:
random_state = 9999
image_output_params = {'width': 1080, 'height': 600, 'scale': 6}
render = 'svg' #or None to have interactive plots

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.inspection import permutation_importance


import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/clean_listingfinal.csv', index_col=0)
# Split data and drop unnecessary data
Y = df['price']
X = df.drop(['price'], axis=1)
X = X.astype('float64')
X.head()
# Split data
XXyy = X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=random_state)
# Storage
result = {'Method':[], 'Train R2':[], 'Train RMSE':[], 'Test R2': [], 'Test RMSE': []}

In [4]:
f"The standard deviation of price of listing is ${df['price'].std():.2f}."

'The standard deviation of price of listing is $132.36.'

In [5]:
# """ Helper functions """
def update_frame(frame: dict, y_train, y_train_pred, y_true, y_pred, method_name):
    frame['Method'].append(method_name)
    frame['Train R2'].append(r2_score(y_train, y_train_pred))
    frame['Train RMSE'].append(np.sqrt(mean_squared_error(y_train, y_train_pred)))
    frame['Test R2'].append(r2_score(y_true, y_pred))
    frame['Test RMSE'].append(np.sqrt(mean_squared_error(y_true, y_pred)))

def plot(y_true, y_pred, title: str, band=True):
    """Plot predicted value against actual value, with formatting added to figure

    Keyword arguments:
    y_true: list like, x axis
    y_pred: list like, y axis
    title: string, added to top-center of figure
    band: boolean, If true, add two band lines to the figure with RMSE distance from y=x line
    """
    h = pd.DataFrame(data={'Actual y':y_true, 'Predicted y': y_pred})
    
    fig = go.Figure()
    fig = px.scatter(h, x='Actual y', y='Predicted y', title=title)
    fig.add_trace(go.Scatter(x=[0, 600],y=[0, 600],mode="lines",line=go.scatter.Line(color='gray'),showlegend=False)) #Diagonal Line
    if band:
        RMSE = df['price'].std()
        fig.add_traces([
            go.Scatter(x=[0, 600],y=[0+RMSE, 600+RMSE],mode='lines',line=go.scatter.Line(dash='dot'),showlegend=False),
            go.Scatter(x=[0, 600],y=[0-RMSE, 600-RMSE],mode='lines',line=go.scatter.Line(dash='dot'),showlegend=False)
        ])
    fig.update_layout(width=700,height=500)
        
    pio.show(fig)
    return fig

def simulate(model, XXyy, name: str):
    """Simulate performance metrics of model using R2 and RMSE and update global result and save plot image

    Keyword arguments:
    model: Implements Skleanr API and must be fitted
    XXyy: output of train_test_split
    """
    # Model must be fitted and implement predict, score methods.
    # XXyy: X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = XXyy
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    update_frame(result, y_train, y_train_pred, y_test, y_pred, name)

    print("-"*30)
    print(f"Train Score: {r2_score(y_train, y_train_pred): .3f}")
    print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)): .3f}")

    print("-"*30)
    print(f"Test Score: {r2_score(y_test, y_pred): .3f}")
    print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)): .3f}")
    
    fig = plot(y_test, y_pred, name)
    # fig.write_image(f'images/{name}.png', **image_output_params)

def feature_plot(feature_name, feature_score, name: str):
    feature_importance = pd.DataFrame({'Feature': feature_name, 'Score': feature_score})
    feature_importance.sort_values(by='Score', axis=0, inplace=True, ascending=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=feature_importance.head(15)['Score'], y=feature_importance.head(15)['Feature'], orientation='h'))
    fig.update_layout(title={'text': f'{name} Most Relevant Feature'}, xaxis_title='Score', yaxis_title='Feature', height=500, width=700)
    pio.show(fig)
    # fig.write_image(f'images/{name} Most Relevant Feature.png', **image_output_params)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=feature_importance.tail(15)['Score'], y=feature_importance.tail(15)['Feature'], orientation='h'))
    fig.update_layout(title={'text': f'{name} Least Relevant Feature'}, xaxis_title='Score', yaxis_title='Feature', height=500, width=700)
    pio.show(fig)
    # fig.write_image(f'images/{name} Least Relevant Feature.png', **image_output_params)

---
<a id='model'></a>
## 1.  Modelling
To find a model that best predicts price. We will being trying :
- linear model: [Linear Regression](#model-1-linear-regression)
- tree-based model: [HistGradientBoostRegressor](#model-2-gradient-boost-regressor), [XGBoost](#model-3-xgboost-regression)
- and to try something new: [Neural Network](#model-4-neural-networks).

We will be using $R^2$ score to compare the goodness of the model, and RMSE to find the accuracy of the prediction.

### Model 1: Linear Regression
We first use linear regression, a method taught for predicting numerical values.

In [6]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

simulate(model, XXyy, 'Linear Regression')
feature_plot([col for col in X.columns], [model.coef_[i].round(5) for i in range(len(X.columns))], 'Linear Regression')

------------------------------
Train Score:  0.691
Train RMSE:  74.117
------------------------------
Test Score:  0.688
Test RMSE:  71.665


### Model 2: Gradient Boost Regressor
We can use gradient boosting - a type of ensemble machine learning algorithms. Ensemble is a collection of decision trees. Every iteration, a decision tree is added to model to minimise the error. Models are fit using any arbitrary differentiable loss function and gradient descent optimization algorithm, and the goal is to minimise loss gradient. We will be using HistGradientBoostingRegressor from sklearn.

In [7]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(random_state=random_state, max_iter=1000)

model.fit(X_train, y_train)

simulate(model, XXyy, name='Gradient Boost Regressor')

feature_plot(model.feature_names_in_, 
             permutation_importance(model, X_test, y_test, random_state=random_state).importances_mean, 
             'Gradient Boost Regressor')

------------------------------
Train Score:  1.000
Train RMSE:  1.433
------------------------------
Test Score:  0.857
Test RMSE:  48.479


### Model 3: XGBoost Regression
Similarly to gradient boosting, we can use XGBoost Regressor to model price. We will use XGBoost API to do the model fitting.

In [8]:
import xgboost as xgb

model = xgb.sklearn.XGBRegressor(tree_method='hist', objective='reg:squarederror', n_estimators = 2500,
                         learning_rate=0.01, random_state=random_state)

model.fit(X_train, y_train)

simulate(model, XXyy, 'XGBoost Regressor')

booster = model.get_booster()
feature_plot(booster.get_score().keys(), booster.get_score().values(), 'XGBoost Regressor')

------------------------------
Train Score:  0.994
Train RMSE:  10.087
------------------------------
Test Score:  0.863
Test RMSE:  47.465


### Model 4: Neural Networks

We will be using Sklearn MLPRegressor, short for Multi-layer Perception Regressor. Model optimizes for lowest squarer error by gradient descent.

In [9]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(random_state=random_state, max_iter=1000).fit(X_train, y_train)

simulate(model, XXyy, 'MLP Regressor')

------------------------------
Train Score:  0.836
Train RMSE:  54.006
------------------------------
Test Score:  0.805
Test RMSE:  56.712


We can conclude that HistGradientBoostingRegressor and XGBoost Regressor achieve the best results, with the lowest RMSE (~50) and the higher R^2 score (~0.85). All other models achieve decent results, with RMSE around half of standard deviation, and R^2 > 0.5.

---
<a id='optimize'></a>
## 2. Optimising models

We will be using RandomizedSearchCV, GridSearchCV, and Hyperopt to find the optimal parameters.
- RandomizedSearchCV is not exhuastive but covers a wide range of hyperparameters.
- GridSearchCV is exhuastive but only convers the range of hyperparameters provided.
- Hyperopt uses TPE algorithm to try hyperparameters values more likely to obtain a better result.

In [10]:
from scipy.stats import loguniform
# Code from scikit-learn-mooc
class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

### Tuning GradientBoostingRegressor with RandomizedSearchCV
Parameters to tune are obtained by consulting the documentation.

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

model = HistGradientBoostingRegressor(random_state=random_state)

param_distributions = {
    'max_bins': loguniform_int(2, 255),
    'max_leaf_nodes': loguniform_int(2, 256),
    'min_samples_leaf': loguniform_int(1, 100),
    'learning_rate': loguniform(0.001, 10),
}

cv = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=350, cv=3, n_jobs=-1, scoring=['neg_mean_squared_error', 'r2'], refit='r2') .fit(X_train, y_train)

v = pd.DataFrame(cv.cv_results_)
v.sort_values(by='rank_test_r2', inplace=True)
display(v.head(6))

print(f"Best parameters: {cv.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_bins,param_max_leaf_nodes,param_min_samples_leaf,params,split0_test_neg_mean_squared_error,...,split2_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2
63,0.393461,0.024050,0.008386,0.000580,0.073928,140,69,13,"{'learning_rate': 0.07392757142166324, 'max_bi...",-2305.400753,...,-2400.729663,-2496.266049,206.222119,1,0.866935,0.856099,0.852603,0.858546,0.006101,1
213,0.578181,0.013383,0.012752,0.003096,0.046528,81,218,10,"{'learning_rate': 0.046527798074886585, 'max_b...",-2309.469608,...,-2511.006411,-2505.600145,157.979073,2,0.866700,0.860564,0.845832,0.857699,0.008757,2
237,0.179151,0.006532,0.005483,0.001046,0.082927,14,32,4,"{'learning_rate': 0.08292701369533048, 'max_bi...",-2398.143493,...,-2315.029571,-2518.220208,231.089184,3,0.861582,0.853058,0.857865,0.857501,0.003489,3
31,0.296680,0.014138,0.005616,0.001398,0.15088,46,56,1,"{'learning_rate': 0.15088048702434512, 'max_bi...",-2294.504264,...,-2429.916479,-2540.919694,258.708565,4,0.867564,0.850118,0.850811,0.856164,0.008066,4
107,0.212506,0.010961,0.005407,0.000592,0.061229,7,38,11,"{'learning_rate': 0.0612294161872509, 'max_bin...",-2250.038653,...,-2488.459788,-2553.105114,277.633005,6,0.870130,0.848955,0.847216,0.855434,0.010416,5
73,0.471626,0.071041,0.015198,0.003442,0.114382,20,108,12,"{'learning_rate': 0.11438181863266934, 'max_bi...",-2303.130867,...,-2531.190050,-2552.926559,213.385488,5,0.867066,0.853938,0.844593,0.855199,0.009218,6


Best parameters: {'learning_rate': 0.07392757142166324, 'max_bins': 140, 'max_leaf_nodes': 69, 'min_samples_leaf': 13}


In [12]:
model = HistGradientBoostingRegressor(**cv.best_params_, random_state=random_state).fit(X_train, y_train)
simulate(model, XXyy, 'RandomSearch CV Gradient Boost Regressor')

------------------------------
Train Score:  0.986
Train RMSE:  15.529
------------------------------
Test Score:  0.865
Test RMSE:  47.063


### Tuning XGBoost Regressor

Parameters to tune:
- n estimators & learning_rate: Takes longer to achieve same error reduction, however smaller steps taken means we can find the optimal minimum
- max_leaves (default 0): By restricting max leaves, we can reduce overfitting.
- colsample_bytree (default 1): Fraction of columns to be randomly sampled, might reduce overfitting.
- subsample (default 1): Fraction of observations to sample for each tree, lower values reduce overfitting.

In [13]:
model = xgb.XGBRegressor(tree_method='hist', objective='reg:squarederror', seed=random_state)

params_grid = {
    'n_estimators': [1000, 1500],
    'learning_rate': [0.03, 0.05],
    'max_leaves': [0, 20, 40, 60, 80],
    'colsample_bytree': [0.4, 0.6, 0.8],
    'subsample': [0.6, 0.8, 1],
}

cv = GridSearchCV(estimator=model, param_grid=params_grid, cv=3, n_jobs=-1, scoring=['neg_mean_squared_error', 'r2'], refit='r2').fit(X_train, y_train)

v = pd.DataFrame(cv.cv_results_)
v.sort_values(by='rank_test_r2', inplace=True)
display(v.head(6))

print(f"Best parameters: {cv.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_leaves,param_n_estimators,param_subsample,params,...,split2_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2
91,1.319172,0.055914,0.021240,0.003739,0.6,0.05,0,1000,0.8,"{'colsample_bytree': 0.6, 'learning_rate': 0.0...",...,-2416.740526,-2467.198624,183.812138,1,0.868887,0.859688,0.851620,0.860065,0.007054,1
115,1.388869,0.030443,0.023054,0.003221,0.6,0.05,80,1000,0.8,"{'colsample_bytree': 0.6, 'learning_rate': 0.0...",...,-2416.740526,-2467.198624,183.812138,1,0.868887,0.859688,0.851620,0.860065,0.007054,1
94,2.011485,0.028456,0.032201,0.005100,0.6,0.05,0,1500,0.8,"{'colsample_bytree': 0.6, 'learning_rate': 0.0...",...,-2419.069889,-2467.540305,183.796208,3,0.868942,0.859706,0.851477,0.860042,0.007134,3
118,2.107627,0.035913,0.025218,0.000879,0.6,0.05,80,1500,0.8,"{'colsample_bytree': 0.6, 'learning_rate': 0.0...",...,-2419.069889,-2467.540305,183.796208,3,0.868942,0.859706,0.851477,0.860042,0.007134,3
139,2.025280,0.112666,0.021286,0.003012,0.8,0.03,60,1000,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2435.371625,-2469.336590,124.551145,5,0.865132,0.863684,0.850476,0.859764,0.006594,5
142,2.400903,0.067154,0.032125,0.005729,0.8,0.03,60,1500,0.8,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2443.182175,-2471.699064,123.554691,6,0.865130,0.863723,0.849996,0.859617,0.006827,6


Best parameters: {'colsample_bytree': 0.6, 'learning_rate': 0.05, 'max_leaves': 0, 'n_estimators': 1000, 'subsample': 0.8}


In [14]:
model = xgb.XGBRegressor(**cv.best_params_,random_state=random_state).fit(X_train, y_train)
simulate(model, XXyy, 'GridSearch CV XGBoost Regressor')

------------------------------
Train Score:  0.999
Train RMSE:  3.118
------------------------------
Test Score:  0.872
Test RMSE:  45.938


### Tuning both GradientBoostingRegressor and XGBoost Regressor with Hyperopt
Using Hyperopt-Sklearn, which is a wrapper of Hyperopt, a library for Distributed Asynchronous Hyper-parameter Optimization.

In [15]:
from hpsklearn import HyperoptEstimator, hist_gradient_boosting_regressor, xgboost_regression
from hyperopt import tpe

model = HyperoptEstimator(
    regressor=hist_gradient_boosting_regressor('HGBR'),
    preprocessing=[],
    algo=tpe.suggest,
    max_evals=20,
    n_jobs=-1,
    seed=random_state)
model.fit(X_train, y_train)

simulate(model, XXyy, 'Hyperopt Gradient Boost Regressor')

100%|██████████| 20/20 [00:00<00:00,  1.13trial/s, best loss: 0.1698249941057911]
------------------------------
Train Score:  0.899
Train RMSE:  42.269
------------------------------
Test Score:  0.861
Test RMSE:  47.885


In [16]:
model = HyperoptEstimator(
    regressor=xgboost_regression('XGBR'),
    preprocessing=[],
    algo=tpe.suggest,
    max_evals=10,
    n_jobs=-1,
    seed=random_state)
model.fit(X_train, y_train)

simulate(model, XXyy, 'Hyperopt XGBoost Regressor')

100%|██████████| 10/10 [00:01<00:00,  1.96s/trial, best loss: 0.1536674470753513]
------------------------------
Train Score:  0.984
Train RMSE:  16.812
------------------------------
Test Score:  0.864
Test RMSE:  47.234


From the optimization results above, we can see that achieving ~45 RMSE, ~0.87 $R^2$ is the best result we can get.

---
<a id='summary'></a>
## 3. Summary

From the plots and the RMSE, $R^2$ values, we can see that the prediction runs well when $0 \leq price \leq 300$ and as price prediction is less accurate when $300 < price$. One reason for higher prices listing to be inaccurate would be that any speciality of the house, like renovation, is only observable through listing picture and cannot be analyzed numerically or categorically. Further improvements can be made if we attempt to anaylze listing description using sentiment analysis etc.

In [17]:
# Save for analysis
pd.DataFrame(result).to_csv('data/model_results.csv')

In [18]:
observartion = pd.read_csv('data/model_results.csv', index_col=0)
obs_RMSE = observartion.sort_values(by='Test RMSE', ascending=False)
obs_R2 = observartion.sort_values(by='Test R2')

In [19]:
fig = make_subplots(rows=2, cols=1, subplot_titles=('Root Mean Square Error sorted by Test Data', 'R2 sorted by Test Data'))

fig.add_trace(go.Bar(name='Train', x=obs_RMSE['Method'], y=obs_RMSE['Train RMSE']), row=1, col=1)
fig.add_trace(go.Bar(name='Test', x=obs_RMSE['Method'], y=obs_RMSE['Test RMSE']), row=1, col=1)

fig.add_trace(go.Bar(name='Train', x=obs_R2['Method'], y=obs_R2['Train R2']), row=2, col=1)
fig.add_trace(go.Bar(name='Test', x=obs_R2['Method'], y=obs_R2['Test R2']), row=2, col=1)

fig.update_layout(height=800, width=800)
fig.show()